# The point of this is that after running into a few issues we need to rethink a few things.

# 1) we can clean and make the funciton more pretty by passing a model.modules object.

# 2) we can calculate the backprop more efficiently by keeping track of the second to last computation in each layer, and continuing from this place

# 3) we need to be able to manipulate precisely when we use numpy vs pytorch. Pytorch GPU will likely calculate the big matrix multiplies faster, but numpy runs better manipulations and indexes on the CPU. Essentially, there are exactly L very large matrix multiplies, so lets only do that in torch.

In [1]:
import numpy as np
import torch
import random

import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#from ..easy_ntk import calculate_NTK
from einops import rearrange

import time

#import sys
#from pathlib import Path

from numba import njit

#from numba import njit
#from numba.typed import List

from numba import njit

%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
#Numpy CNN function-- checked that it agrees with Tensorflow.
#need to make sure it agrees with PyTorch.


class Conv2d():
    def __init__(self,F,strides=1,padding=0):
        '''
        PYTORCH IMPLEMENTATION!!!
        
        this uses non-contiguous arrays, so likely a better solution exists
        and it can't be sped up with numba and reshapes
        
        #given an input of x = [batches, length, channels]
        
        #F an input of = [channels_out, channels, kernel_size]
        
        #B an input of [channels_out]
        
        #ouputs a shape: [batches, new_length, channels_out]
        
        #   1. Flattens the filter to a 2-D matrix with shape
        #      `[filter_height * filter_width * in_channels, output_channels]`.
        #   2. Extracts image patches from the input tensor to form a *virtual*
        #      tensor of shape `[batch, out_height, out_width, 
        #      filter_height * filter_width * in_channels]`. batch, new_length, kernel_size * in_channels
        #   3. For each patch, right-multiplies the filter matrix and the image patch
        #      vector.
        '''
        if padding < 0:
            raise ValueError('Padding must be a non-negative int')
        
        self.out_filters = F.shape[0]
        self.in_filters = F.shape[1]
        self.kernel_height = F.shape[2]
        self.kernel_width = F.shape[3]
        self.F = F.T #filters array, now is [width, height, channels_in, channels_out]
        #self.B = B #bias array
        self.strides = strides
        self.padding = padding
        
        self.F = np.reshape(self.F,(-1,self.out_filters))
        
    def forward(self,x):
        batches, channels_in, height, width  = np.shape(x)
        
        if self.padding != 0:
            x = np.pad(x,((0,0),(0,0),(self.padding,self.padding),(self.padding,self.padding)),mode='constant',constant_values=0.0)
    
        new_height = int(((height + 2*self.padding - (self.kernel_height))/self.strides) + 1)
        new_width = int(((width + 2*self.padding - (self.kernel_width))/self.strides) + 1)
        
        dumb_array = np.zeros((batches, new_height, new_width, self.kernel_width * self.kernel_height * channels_in),dtype=np.float32)
        
        for i in range(new_height):
            for j in range(new_width):
                dumb_array[:,i,j,:] = np.reshape(x[:,:,self.strides*i:self.strides*i+self.kernel_height, self.strides*j:self.strides*j+self.kernel_width],(batches, self.kernel_width * self.kernel_height * channels_in),order='F')

        output_array = dumb_array @ self.F
        
        output_array = rearrange(output_array,'b h w f -> b f h w')

        return output_array #+ self.B[None,None,:]

In [3]:
import copy

def _del_nested_attr(obj, names):
    """
    Deletes the attribute specified by the given list of names.
    For example, to delete the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'])
    """
    if len(names) == 1:
        delattr(obj, names[0])
    else:
        _del_nested_attr(getattr(obj, names[0]), names[1:])

def _set_nested_attr(obj, names, value):
    """
    Set the attribute specified by the given list of names to value.
    For example, to set the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'], value)
    """
    if len(names) == 1:
        setattr(obj, names[0], value)
    else:
        _set_nested_attr(getattr(obj, names[0]), names[1:], value)

def extract_weights(mod):
    """
    This function removes all the Parameters from the model and
    return them as a tuple as well as their original attribute names.
    The weights must be re-loaded with `load_weights` before the model
    can be used again.
    Note that this function modifies the model in place and after this
    call, mod.parameters() will be empty.
    """
    orig_params = tuple(mod.parameters())
    # Remove all the parameters in the model
    names = []
    for name, p in list(mod.named_parameters()):
        _del_nested_attr(mod, name.split("."))
        names.append(name)

    # Make params regular Tensors instead of nn.Parameter
    params = tuple(p.detach().requires_grad_() for p in orig_params)
    return params, names

def load_weights(mod, names, params):
    """
    Reload a set of weights so that `mod` can be used again to perform a forward pass.
    Note that the `params` are regular Tensors (that can have history) and so are left
    as Tensors. This means that mod.parameters() will still be empty after this call.
    """
    for name, p in zip(names, params):
        _set_nested_attr(mod, name.split("."), p)
        
def calculate_NTK(model,x,device='cpu',MODE='samples'):
    """
    INPUTS:
        model: torch.nn.Module 
        x: torch.Tensor
        device: 'cpu',
        MODE: 'minima'
    
    OUTPUTS:
        NTK: torch.Tensor
    
    Calculates the NTK for a model, p_dict a state dictionary, and x, a single tensor fed into the model
    
    The NTK is the grammian of the Jacobian of the model output to w.r.t. the weights of the model
    
    This function will output the NTK such that the minima matrix size is used. If the Jacobian is an NxM
    matrix, then the NTK is formulated so that if N < M; NTK is NxN. If M<N, then NTK is MxM.
    
    #EXAMPLE USAGE:
    device='cpu'
    model = MODEL() #a torch.nn.Module object 
    model.to(device)
    state_dict = model.state_dict()

    x_test = np.ones((100,1,28,28),dtype=np.float32)
    x_test = torch.from_numpy(x_test)

    NTK = calculate_NTK(model,x_test)
    """
    if not(MODE in ['minima','samples','params']):
        raise ValueError("MODE must be one of 'minima','samples','params'")
    
    x = x.to(device)
    x.requires_grad=False
    N = x.shape[0]
    M = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    #We need to create a clone of the model or else we make it unusable as part of the trickery 
    #to get pytorch to do what we want. Unforutantely, this exlcludes super big models. but, eh.
    model_clone = copy.deepcopy(model)
    
    params, names = extract_weights(model_clone)
    def model_ntk(*args,model=model_clone, names=names):
        params = tuple(args)
        load_weights(model, names, params)
        return model(x)
    
    Js = torch.autograd.functional.jacobian(model_ntk, tuple(params), create_graph=False, vectorize=True)
    
    Js = list(Js)
    #Js = [element for tupl in Js for element in tupl]
    #collapse the tensors
    for i,tensor in enumerate(Js):
        Js[i] = tensor.reshape(N,-1)
    
    J = torch.cat(Js,axis=1)
    
    if MODE=='minima':
        if N < M: #if datasize points is less than number of parameters:
            NTK = torch.matmul(J,J.T)

        if N >= M:#if number of parameters is less than datasize:
            NTK = torch.matmul(J.T,J)
    elif MODE=='samples':
        NTK = torch.matmul(J,J.T)
    elif MODE=='params':
        NTK = torch.matmul(J.T,J)
    
    return NTK

In [4]:
@njit
def zero_pad(A,pad):
    N, F, H, W = A.shape
    P = np.zeros((N, F, H+2*pad, W+2*pad),dtype=np.float32)
    P[:,:,pad:H+pad,pad:W+pad] = A
    return P

In [5]:
@njit
def calc_dw(x,w,b,pad,stride,H_,W_):
    """
    Calculates the derivative of conv(x,w) with respect to w
    
    output is shape:
        [datapoints, in_channels out_filters, kernel_height, kernel_width, out_filters, data_height, data_width
    
    'n f1 f2 c kh kw dh dw -> n (c f1 kh kw) (f2 dh dw)'
    """
    dx, dw, db = None, None, None
    N, C, H, W = x.shape
    F, _, HH, WW = w.shape
    
    #dw = np.zeros((N,F,F,C,HH,WW,H_,W_),dtype=np.float32)
    
    dw = np.zeros((N,C,F,HH,WW,F,H_,W_),dtype=np.float32)

    xp = zero_pad(x,pad)
    #high priority, how to vectorize this operation?
    for n in range(N):
        for f in range(F):
            for i in range(HH): 
                for j in range(WW): 
                    for k in range(H_): 
                        for l in range(W_): 
                            for c in range(C): 
                                dw[n,c,f,i,j,f,k,l] += xp[n, c, i+stride*k, j+stride*l]                             
    
    return dw.reshape((N,(C*F*HH*WW),(F*H_*W_)))

@njit
def calc_dx(x,w,b,pad,stride,H_,W_):
    '''
    calculates the derivative of conv(x,w) with respect to x
    
    output is a nd-array of shape n x ch_in x og_h x og_w x (h_out w_out ch_out)
    '''
    dx, dw, db = None, None, None
    N, C, H, W = x.shape
    F, _, HH, WW = w.shape 

    dx = np.zeros((C,H,W,F,H_,W_,),dtype=np.float32)
    #high priority, how to vectorize this operation? maybe with np.chunk,split?
    for f in range(F): 
        for i in range(H): 
            for j in range(W):
                for k in range(H_): 
                    for l in range(W_):
                        for c in range(C): 
                            if i-stride*k+pad > HH-1 or j-stride*l+pad > WW-1:
                                continue #this is alternative to padding w with zeros.
                            if i-stride*k+pad < 0 or j-stride*l+pad < 0:
                                continue #this is alternative to padding w with zeros.
                            dx[c,i,j,f,k,l] += w[f, c, i-stride*k+pad, j-stride*l+pad]
    #'c ih iw f oh ow -> (c ih iw) (f oh ow)'
    return dx.reshape(((C*H*W),(F*H_*W_)))


In [6]:
#%%timeit

#can I speed this up with chunks?
#l=1
#5.9ms vs 20.8 micro seconds; numba is taking care of the for loop vectorization for us.
#for numpy this seems fine. 
#calc_dx(Xs[l].T,w=Ks[l],b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])

In [7]:
def relu(X,normalize=False):
    X = F.relu(X)
    if normalize:
        return np.sqrt(2*np.pi/(np.pi-1))*(X-1/np.sqrt(2*np.pi))
    else:
        return X
    

# #Identity
# def activation(x):
#     return x

# @njit
# def d_activation(x):
#     return np.ones(np.shape(x),dtype=np.float32) 


#Tanh
def activation(x):
    return torch.tanh(x)

@njit
def d_activation(x):
    return np.cosh(x)**-2

In [8]:
SEED = 0

In [9]:
def NTK_weights(m):
    if isinstance(m, nn.Linear):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]
    if isinstance(m, nn.Conv2d):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]

In [10]:
# Easy NTK expects one output alone
class dumb_small(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small, self).__init__()
        
        self.d1 = torch.nn.Conv2d(1,4,3,bias=True)
        
        self.d2 = torch.nn.Conv2d(4,4,3,stride=2,padding=1,bias=True)

        self.d3 = torch.nn.Conv2d(4,6,3,stride=1,padding=0,bias=True)
        
        self.d4 = torch.nn.Conv2d(6,8,3,stride=1,padding=0,bias=True)
        
        self.d5 = torch.nn.Conv2d(8,8,7,stride=1,padding=0,bias=True)
        
        self.d6 = torch.nn.Linear(3*3*8,16,bias=True)
        
        self.d7 = torch.nn.Linear(16,1,bias=True)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0))
        x_2 = activation(self.d2(x_1))
        x_3 = activation(self.d3(x_2))
        x_4 = activation(self.d4(x_3))
        x_5 = activation(self.d5(x_4))
        x_6 = x_5.reshape(-1,3*3*8)
        x_7 = activation(self.d6(x_6))
        x_8 = self.d7(x_7)
        return x_8
    
# Easy NTK expects one output alone
class dumb_small_layerwise(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small_layerwise, self).__init__()
        
        self.d1 = torch.nn.Conv2d(1,4,3,bias=True)#28 -> 26

        self.d2 = torch.nn.Conv2d(4,4,3,stride=2,padding=1,bias=True)#26 -> 13
        
        self.d3 = torch.nn.Conv2d(4,6,3,stride=1,padding=0,bias=True)#13 -> 11
        
        self.d4 = torch.nn.Conv2d(6,8,3,stride=1,padding=0,bias=True)#11 -> 9
        
        self.d5 = torch.nn.Conv2d(8,8,7,stride=1,padding=0,bias=True)#9 -> 3
        
        self.d6 = torch.nn.Linear(3*3*8,16,bias=True)
        
        self.d7 = torch.nn.Linear(16,1,bias=True)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0))
        x_2 = activation(self.d2(x_1))
        x_3 = activation(self.d3(x_2))
        x_4 = activation(self.d4(x_3))
        x_5 = activation(self.d5(x_4))
        x_6 = x_5.reshape(-1,3*3*8)
        x_7 = activation(self.d6(x_6))
        x_8 = self.d7(x_7)
        return x_8, x_7, x_6, x_5, x_4, x_3, x_2, x_1, x_0
    

In [11]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_small = dumb_small()
model_small.to(device)
model_small.apply(NTK_weights)

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_layerwise = dumb_small_layerwise()
model_layerwise.to(device)
model_layerwise.apply(NTK_weights)

x_test = np.random.normal(0,1,(300,1,28,28)).astype(np.float32) #n c_in, h, w
x_test = torch.from_numpy(x_test)

torch.Size([4, 1, 3, 3])
torch.Size([4, 4, 3, 3])
torch.Size([6, 4, 3, 3])
torch.Size([8, 6, 3, 3])
torch.Size([8, 8, 7, 7])
torch.Size([16, 72])
torch.Size([1, 16])
torch.Size([4, 1, 3, 3])
torch.Size([4, 4, 3, 3])
torch.Size([6, 4, 3, 3])
torch.Size([8, 6, 3, 3])
torch.Size([8, 8, 7, 7])
torch.Size([16, 72])
torch.Size([1, 16])


In [15]:
assert torch.all(model_layerwise.d1.weight == model_small.d1.weight)
assert torch.all(model_layerwise.d2.weight == model_small.d2.weight)

# Easy_NTK

In [13]:
#NTK_easy = calculate_NTK(model_small,x_test).detach().numpy()

KeyboardInterrupt: 

# Pytorch Autograd

In [12]:
model_small.zero_grad()
y = model_small(x_test)

In [13]:
#this method agrees between model layerwise and model small; meaning that the calculation is indepdent of those
#two models. the insinuation is somehting is wrong with both my methods for calculating,--- the same thing, since
#they agree with one another.

#in the future we would iterate over layers instead of like this...
layer_components_w1 = [] 
layer_components_w2 = []
layer_components_w3 = []
layer_components_w4 = []
layer_components_w5 = []
layer_components_w6 = []
layer_components_w7 = []

layer_components_b1 = []
layer_components_b2 = []
layer_components_b3 = []
layer_components_b4 = []
layer_components_b5 = []
layer_components_b6 = []
layer_components_b7 = []

for output in y:
    model_small.zero_grad()
    
    output.backward(retain_graph=True)

    #Get the tensors
    w1_grad = model_small.d1.weight.grad.detach().numpy()
    w2_grad = model_small.d2.weight.grad.detach().numpy()
    w3_grad = model_small.d3.weight.grad.detach().numpy()
    w4_grad = model_small.d4.weight.grad.detach().numpy()
    w5_grad = model_small.d5.weight.grad.detach().numpy()
    w6_grad = model_small.d6.weight.grad.detach().numpy()
    w7_grad = model_small.d7.weight.grad.detach().numpy()
    
    b1_grad = model_small.d1.bias.grad.detach().numpy()
    b2_grad = model_small.d2.bias.grad.detach().numpy()
    b3_grad = model_small.d3.bias.grad.detach().numpy()
    b4_grad = model_small.d4.bias.grad.detach().numpy()
    b5_grad = model_small.d5.bias.grad.detach().numpy()
    b6_grad = model_small.d6.bias.grad.detach().numpy()
    b7_grad = model_small.d7.bias.grad.detach().numpy()

    #reshape and append. deep copy neccessary or else they are the same objects
    layer_components_w1.append(w1_grad.reshape(-1).copy())
    layer_components_w2.append(w2_grad.reshape(-1).copy())
    layer_components_w3.append(w3_grad.reshape(-1).copy())
    layer_components_w4.append(w4_grad.reshape(-1).copy())
    layer_components_w5.append(w5_grad.reshape(-1).copy())
    layer_components_w6.append(w6_grad.reshape(-1).copy())
    layer_components_w7.append(w7_grad.reshape(-1).copy())
    
    layer_components_b1.append(b1_grad.reshape(-1).copy())
    layer_components_b2.append(b2_grad.reshape(-1).copy())
    layer_components_b3.append(b3_grad.reshape(-1).copy())
    layer_components_b4.append(b4_grad.reshape(-1).copy())
    layer_components_b5.append(b5_grad.reshape(-1).copy())
    layer_components_b6.append(b6_grad.reshape(-1).copy())
    layer_components_b7.append(b7_grad.reshape(-1).copy())

In [14]:
layer_components_w1 = np.array(layer_components_w1)
layer_components_w2 = np.array(layer_components_w2)
layer_components_w3 = np.array(layer_components_w3)
layer_components_w4 = np.array(layer_components_w4)
layer_components_w5 = np.array(layer_components_w5)
layer_components_w6 = np.array(layer_components_w6)
layer_components_w7 = np.array(layer_components_w7)

layer_components_b1 = np.array(layer_components_b1)
layer_components_b2 = np.array(layer_components_b2)
layer_components_b3 = np.array(layer_components_b3)
layer_components_b4 = np.array(layer_components_b4)
layer_components_b5 = np.array(layer_components_b5)
layer_components_b6 = np.array(layer_components_b6)
layer_components_b7 = np.array(layer_components_b7)

In [15]:
autograd_NTK = layer_components_w1 @ layer_components_w1.T+\
    layer_components_w2 @ layer_components_w2.T+\
    layer_components_w3 @ layer_components_w3.T+\
    layer_components_w4 @ layer_components_w4.T+\
    layer_components_w5 @ layer_components_w5.T+\
    layer_components_w6 @ layer_components_w6.T+\
    layer_components_w7 @ layer_components_w7.T+\
    layer_components_b1 @ layer_components_b1.T+\
    layer_components_b2 @ layer_components_b2.T+\
    layer_components_b3 @ layer_components_b3.T+\
    layer_components_b4 @ layer_components_b4.T+\
    layer_components_b5 @ layer_components_b5.T+\
    layer_components_b6 @ layer_components_b6.T+\
    layer_components_b7 @ layer_components_b7.T

# Layerwise

In [61]:
model_layerwise.to('cpu')
x_8, x_7, x_6, x_5, x_4, x_3, x_2, x_1, x_0 = model_layerwise(x_test)

#Dense Weight Matrices
Ws = []
Ws.append(np.array([0.0],dtype=np.float32)) #spacer
Ws.append(np.array([0.0],dtype=np.float32)) #spacer 
Ws.append(np.array([0.0],dtype=np.float32))
Ws.append(np.array([0.0],dtype=np.float32))
Ws.append(np.array([0.0],dtype=np.float32)) 
Ws.append(np.array([0.0],dtype=np.float32)) #spacer reshape is a layer
Ws.append(model_layerwise.d6.weight.detach().numpy().astype(np.float32))
Ws.append(model_layerwise.d7.weight.detach().numpy().astype(np.float32))

Wspt = []
Wspt.append(np.array([0.0],dtype=np.float32)) #spacer
Wspt.append(np.array([0.0],dtype=np.float32)) #spacer 
Wspt.append(np.array([0.0],dtype=np.float32))
Wspt.append(np.array([0.0],dtype=np.float32))
Wspt.append(np.array([0.0],dtype=np.float32)) 
Wspt.append(np.array([0.0],dtype=np.float32)) #spacer reshape is a layer
Wspt.append(model_layerwise.d6.weight.detach())
Wspt.append(model_layerwise.d7.weight.detach())


#Kernel Matrices
Ks = []
Ks.append(model_layerwise.d1.weight.detach().numpy().astype(np.float32))
Ks.append(model_layerwise.d2.weight.detach().numpy().astype(np.float32))
Ks.append(model_layerwise.d3.weight.detach().numpy().astype(np.float32))
Ks.append(model_layerwise.d4.weight.detach().numpy().astype(np.float32))
Ks.append(model_layerwise.d5.weight.detach().numpy().astype(np.float32))
Ks.append(np.array([0.0],dtype=np.float32)) #spacer
Ks.append(np.array([0.0],dtype=np.float32)) #spacer
Ks.append(np.array([0.0],dtype=np.float32)) #spacer

Kspt = []
Kspt.append(model_layerwise.d1.weight.detach())
Kspt.append(model_layerwise.d2.weight.detach())
Kspt.append(model_layerwise.d3.weight.detach())
Kspt.append(model_layerwise.d4.weight.detach())
Kspt.append(model_layerwise.d5.weight.detach())
Kspt.append(np.array([0.0],dtype=np.float32)) #spacer
Kspt.append(np.array([0.0],dtype=np.float32)) #spacer
Kspt.append(np.array([0.0],dtype=np.float32)) #spacer

# Xs = [] # Xs are shape (output x #DP) ; however, typical python notation is reversed, so we take transpose here
# Xs.append(x_0.detach().numpy().T.astype(np.float32))
# Xs.append(x_1.detach().numpy().T.astype(np.float32))
# Xs.append(x_2.detach().numpy().T.astype(np.float32))
# Xs.append(x_3.detach().numpy().T.astype(np.float32))
# Xs.append(x_4.detach().numpy().T.astype(np.float32))
# Xs.append(x_5.detach().numpy().T.astype(np.float32))
# Xs.append(x_6.detach().numpy().T.astype(np.float32))
# Xs.append(x_7.detach().numpy().T.astype(np.float32))

Xs = [] # Xs are shape (output x #DP) ; however, typical python notation is reversed, so we take transpose here
Xs.append(x_0.T.detach())
Xs.append(x_1.T.detach())
Xs.append(x_2.T.detach())
Xs.append(x_3.T.detach())
Xs.append(x_4.T.detach())
Xs.append(x_5.T.detach())
Xs.append(x_6.T.detach())
Xs.append(x_7.T.detach())


Bs = []
Bs.append(model_layerwise.d1.bias.detach().numpy().astype(np.float32))
Bs.append(model_layerwise.d2.bias.detach().numpy().astype(np.float32))
Bs.append(model_layerwise.d3.bias.detach().numpy().astype(np.float32))
Bs.append(model_layerwise.d4.bias.detach().numpy().astype(np.float32))
Bs.append(model_layerwise.d5.bias.detach().numpy().astype(np.float32))
Bs.append(np.array([0.0],dtype=np.float32))
Bs.append(model_layerwise.d6.bias.detach().numpy().astype(np.float32)[:,None])
Bs.append(model_layerwise.d7.bias.detach().numpy().astype(np.float32)[:,None])

#This is used to create arrays-- needs to be integer list to play nice with compilers
ds_int = []
ds_int.append(0)
ds_int.append(4*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(4*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(6*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*7*7) #channels_out * kernel_height * kernel_width
ds_int.append(16) #number output features
ds_int.append(1) #number output features

ds_array = [] #this is for the NTK formulation, 
ds_array.append(np.array([1.0],dtype=np.float32)) #first element is a spacer, could be anything.
ds_array.append(np.array([1.0],dtype=np.float32)) #The rest, even if you dont use NTK formulation, would be 1
ds_array.append(np.array([1.0],dtype=np.float32))
ds_array.append(np.array([1.0],dtype=np.float32))
ds_array.append(np.array([1.0],dtype=np.float32))
ds_array.append(np.array([1.0],dtype=np.float32))
ds_array.append(np.array([1.0],dtype=np.float32))
ds_array.append(np.array([1.0],dtype=np.float32))

padding = []
padding.append(0)
padding.append(1)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)

strides = []
strides.append(1)
strides.append(2)
strides.append(1)
strides.append(1)
strides.append(1)
strides.append(0)
strides.append(0)
strides.append(0)

layers =[model_layerwise.d1,
         model_layerwise.d2,
         model_layerwise.d3,
         model_layerwise.d4,
         model_layerwise.d5,
         0.0,
         model_layerwise.d6,
         model_layerwise.d7]

# Improve this algorithm's speed: here is the original

In [62]:
@njit
def cross(X):
    return X.T.dot(X)

def cross_pt_nonp(X,device='cuda'):
    X = X.to(device)
    return X.T.matmul(X).cpu()

def cross_pt(X,device='cuda'):
    X = torch.from_numpy(X).to(device)
    X = X.to(device)
    return X.T.matmul(X).cpu()

def d_activationt(x):
    return torch.cosh(x)**-2

In [63]:
#here are the updates we plan:
def compute_NTK_CNN_w_Bias_oneshot(Ws, Ks, Xs, Bs, d_int, d_array, strides, padding, layers, device="cpu"):
    components = []
    
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK
    n = Xs[0].shape[-1] #number of datapoints

    #holds the derivatives of activation, first value is empty list...?; just a spacer, replace with array
    Ds_dense = [np.array([[0.0]],dtype=np.float32)] 
    Ds_conv = [np.array([[0.0]],dtype=np.float32)]
    dws = []
    dxs = []
    s_matrices = []
    with torch.no_grad():
        ####################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Linear):
                Ds_dense.append(d_activationt(layers[l](Xs[l].T)).T.numpy())
            else:
                Ds_dense.append(np.array([[0.0]],dtype=np.float32))
        ################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Conv2d):
                Ds_conv.append(d_activationt(layers[l](Xs[l].T)).numpy().reshape(n,-1).T)
            else:
                Ds_conv.append(np.array([[0.0]],dtype=np.float32))
        ####################################################################################################        
        for l in range(0,L):

            if np.all(Ks[l]!=0):
                dw = calc_dw(x=Xs[l].T.numpy(),w=Ks[l],b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                dws.append(dw)
                if l != 0:
                    dx = calc_dx(x=Xs[l].T.numpy(),w=Ks[l],b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                    dxs.append(dx[None,:,:])
                else:
                    dxs.append(np.array([[0.0]],dtype=np.float32))
            else:
                dws.append(np.array([[0.0]],dtype=np.float32))
                dxs.append(np.array([[0.0]],dtype=np.float32))
        
        
        ####################################################################################################
        S = np.array([1.0],dtype=np.float32) #this models the backward propogation:   
        for l in range(L,-1,-1):
            #first we append the layers end to our component list
            if isinstance(layers[l], torch.nn.Linear):

                components.append(cross_pt(S,device)*cross_pt_nonp(Xs[l],device))

                W = np.multiply(np.ones((d_int[l],n),dtype=np.float32),S)
                components.append(cross_pt(W,device))

            elif isinstance(layers[l], torch.nn.Conv2d):
                if len(S.shape) == 2: #this should only affect the very last layer, at which point, who cares.
                    S = S[None,:,:]
                    
                #s_matrices.append(S)
                W = (torch.from_numpy(dws[l]).to(device) @ torch.from_numpy(S).to(device)).cpu().numpy()
                #W = torch.matmul(dws[l].to_sparse(),torch.from_numpy(S).to(device)).cpu().numpy() 
                
                #We should bring this to zhichao or sombody and ask if there is obviously something faster?
                W = np.diagonal(W,0,2,0)

                components.append(cross_pt(W,device))

                N = Ks[l].shape[0]
                W = np.split(S,N,axis=1)
                W = np.array(W)
                W = np.sum(W,axis=(1,2))
                components.append(cross_pt(W,device))

            #############################
            #now we setup S for the next loop by treating appropriately
            if l==0:
                break

            if isinstance(layers[l], torch.nn.Linear):
                S = np.dot(S.T,Ws[l]).T/np.sqrt(d_array[l])
                if len(S.shape) < 2:
                    S = S[:,None] #expand dimension along axis 1
                if not(isinstance(layers[l-1],float)): #this exludes the reshaping layer
                    S = Ds_dense[l]*S
                else: #and when the reshaping layer occurs we need to apply this instead
                    S = Ds_conv[l-1]*S

            elif isinstance(layers[l], torch.nn.Conv2d):
                S = (dxs[l] @ S) / np.sqrt(d_array[l])
                S = Ds_conv[l]*S
            
    return components#, dws, s_matrices

In [64]:
#here are the updates we plan:
def compute_NTK_CNN_w_Bias_oneshot_asneeded(Ws, Ks, Xs, Bs, d_int, d_array, strides, padding, layers, device="cpu"):
    components = []
    
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK
    n = Xs[0].shape[-1] #number of datapoints

    #holds the derivatives of activation, first value is empty list...?; just a spacer, replace with array
    Ds_dense = [np.array([[0.0]],dtype=np.float32)] 
    Ds_conv = [np.array([[0.0]],dtype=np.float32)]
    s_matrices = []
    with torch.no_grad():
        ####################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Linear):
                Ds_dense.append(d_activationt(layers[l](Xs[l].T)).T.numpy())
            else:
                Ds_dense.append(np.array([[0.0]],dtype=np.float32))
        ################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Conv2d):
                Ds_conv.append(d_activationt(layers[l](Xs[l].T)).numpy().reshape(n,-1).T)
            else:
                Ds_conv.append(np.array([[0.0]],dtype=np.float32))      
        ####################################################################################################
        S = np.array([1.0],dtype=np.float32) #this models the backward propogation:   
        for l in range(L,-1,-1):
            if isinstance(layers[l], torch.nn.Linear):

                components.append(cross_pt(S,device)*cross_pt_nonp(Xs[l],device))

                W = np.multiply(np.ones((d_int[l],n),dtype=np.float32),S)
                components.append(cross_pt(W,device))

            elif isinstance(layers[l], torch.nn.Conv2d):
                if len(S.shape) == 2: #this should only affect the very last layer, at which point, who cares.
                    S = S[None,:,:]
                    
                dw = calc_dw(x=Xs[l].T.numpy(),w=Ks[l],b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                W = (torch.from_numpy(dw).to(device) @ torch.from_numpy(S).to(device)).cpu().numpy()
                
                #We should bring this to zhichao or sombody and ask if there is obviously something faster?
                W = np.diagonal(W,0,2,0)

                components.append(cross_pt(W,device))

                N = Ks[l].shape[0]
                W = np.split(S,N,axis=1)
                W = np.array(W)
                W = np.sum(W,axis=(1,2))
                components.append(cross_pt(W,device))

            #############################
            #now we setup S for the next loop by treating appropriately
            if l==0:
                break

            if isinstance(layers[l], torch.nn.Linear):
                S = np.dot(S.T,Ws[l]).T/np.sqrt(d_array[l])
                if len(S.shape) < 2:
                    S = S[:,None] #expand dimension along axis 1
                if not(isinstance(layers[l-1],float)): #this exludes the reshaping layer
                    S = Ds_dense[l]*S
                else: #and when the reshaping layer occurs we need to apply this instead
                    S = Ds_conv[l-1]*S

            elif isinstance(layers[l], torch.nn.Conv2d):
                dx = calc_dx(x=Xs[l].T.numpy(),w=Ks[l],b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                S = (dx[None,:,:] @ S) / np.sqrt(d_array[l])
                S = Ds_conv[l]*S
            
    return components#, dws, s_matrices

In [100]:
#here are the updates we plan:
def compute_NTK_CNN_w_Bias_oneshot_PT(Ws, Ks, Xs, Bs, d_int, d_array, strides, padding, layers, device="cpu"):
    components = []
    
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK
    n = Xs[0].shape[-1] #number of datapoints

    #holds the derivatives of activation, first value is empty list...?; just a spacer, replace with array
    Ds_dense = [np.array([[0.0]],dtype=np.float32)] 
    Ds_conv = [np.array([[0.0]],dtype=np.float32)]
    s_matrices = []
    with torch.no_grad():
        ####################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Linear):
                Ds_dense.append(d_activationt(layers[l](Xs[l].T)).T)
            else:
                Ds_dense.append(np.array([[0.0]],dtype=np.float32))
        ################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Conv2d):
                Ds_conv.append(d_activationt(layers[l](Xs[l].T)).reshape(n,-1).T)
            else:
                Ds_conv.append(np.array([[0.0]],dtype=np.float32))      
        ####################################################################################################
        S = torch.tensor([1.0],dtype=torch.float32) #this models the backward propogation:   
        for l in range(L,-1,-1):
            if isinstance(layers[l], torch.nn.Linear):

                components.append(cross_pt_nonp(S,device)*cross_pt_nonp(Xs[l],device))

                W = torch.ones((d_int[l],n),dtype=torch.float32) * S
                components.append(cross_pt_nonp(W,device))

            elif isinstance(layers[l], torch.nn.Conv2d):
                if len(S.shape) == 2: #this should only affect the very last layer, at which point, who cares.
                    S = S[None,:,:]
                    
                dw = calc_dw(x=Xs[l].T.numpy(),w=Ks[l].numpy(),b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                
                W = torch.matmul(torch.from_numpy(dw).to(device),S.to(device)).cpu()
                
                #We should bring this to zhichao or sombody and ask if there is obviously something faster?
                #W = np.diagonal(W,0,2,0)
                W = torch.diagonal(W,0,0,2)

                components.append(cross_pt_nonp(W,device))

                N = Ks[l].shape[0]
                #W = np.split(S,N,axis=1)
                W = torch.split(S,N,dim=1)
                
                #W = np.array(W)
                W = torch.stack(W)
                
                #W = np.sum(W,axis=(1,2))
                W = torch.sum(W,dim=(1,2))

                components.append(cross_pt_nonp(W,device))

            #############################
            #now we setup S for the next loop by treating appropriately
            if l==0:
                break

            if isinstance(layers[l], torch.nn.Linear):
                #S = np.dot(S.T,Ws[l]).T/np.sqrt(d_array[l])
                S = torch.matmul(S.T,Ws[l]) / np.sqrt(d_array[l])
                if len(S.shape) < 2:
                    S = S[:,None] #expand dimension along axis 1
                if not(isinstance(layers[l-1],float)): #this exludes the reshaping layer
                    S = Ds_dense[l]*S
                else: #and when the reshaping layer occurs we need to apply this instead
                    S = Ds_conv[l-1]*S.T

            elif isinstance(layers[l], torch.nn.Conv2d):
                dx = calc_dx(x=Xs[l].T.numpy(),w=Ks[l].numpy(),b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                S = (torch.from_numpy(dx[None,:,:]) @ S) / np.sqrt(d_array[l])
                S = Ds_conv[l]*S
            
    return components#, dws, s_matrices

In [101]:
#torch.cuda.empty_cache()

In [103]:
%%timeit
components = compute_NTK_CNN_w_Bias_oneshot_asneeded(Wspt, Kspt, Xs, Bs, ds_int, ds_array, strides, padding, layers, device='cuda')

1.15 s ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%lprun -f compute_NTK_CNN_w_Bias_oneshot_PT compute_NTK_CNN_w_Bias_PT(Ws, Ks, Xs, Bs, ds_int, ds_array, strides, padding, layers, device="cuda")

NameError: name 'compute_NTK_CNN_w_Bias_PT' is not defined

In [64]:
#5.94 is fast compared to the time of easy_ntk.

In [65]:
sum(p.numel() for p in model_layerwise.parameters() if p.requires_grad)
#lol yikes

5179

# Compare Against One another

In [37]:
NTK_layerwise = np.sum(components)
NTK_layerwise.cpu().numpy()

C:\Users\enge625\Miniconda3\envs\pytorch1.9\lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\enge625\Miniconda3\envs\pytorch1.9\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtyp

array([[ 1.3348228e+05,  2.5838447e+03, -5.5709082e+03, ...,
         1.5912437e+03, -8.4559541e+03, -1.3930099e+02],
       [ 2.5838447e+03,  1.3920228e+03,  5.7449066e+01, ...,
        -1.6074094e+03, -2.0567888e+02,  6.2342400e+01],
       [-5.5709082e+03,  5.7449066e+01,  2.4104357e+04, ...,
        -4.5882139e+03,  8.9728925e+02,  2.1850346e+01],
       ...,
       [ 1.5912437e+03, -1.6074094e+03, -4.5882139e+03, ...,
         5.2759320e+04, -1.9727719e+03, -1.2601754e+03],
       [-8.4559541e+03, -2.0567888e+02,  8.9728925e+02, ...,
        -1.9727719e+03,  5.9253203e+03,  5.6632980e+01],
       [-1.3930099e+02,  6.2342400e+01,  2.1850346e+01, ...,
        -1.2601754e+03,  5.6632980e+01,  4.5298846e+02]], dtype=float32)

In [49]:
NTK_easy

array([[ 1.3348217e+05,  2.5838464e+03, -5.5709131e+03],
       [ 2.5838467e+03,  1.3920264e+03,  5.7448795e+01],
       [-5.5709131e+03,  5.7448689e+01,  2.4104338e+04]], dtype=float32)

In [38]:
autograd_NTK

array([[ 1.3348231e+05,  2.5838425e+03, -5.5709087e+03, ...,
         1.5912489e+03, -8.4559551e+03, -1.3929858e+02],
       [ 2.5838425e+03,  1.3920225e+03,  5.7448956e+01, ...,
        -1.6074094e+03, -2.0567831e+02,  6.2342556e+01],
       [-5.5709087e+03,  5.7448956e+01,  2.4104354e+04, ...,
        -4.5882158e+03,  8.9729010e+02,  2.1850412e+01],
       ...,
       [ 1.5912489e+03, -1.6074094e+03, -4.5882158e+03, ...,
         5.2759348e+04, -1.9727728e+03, -1.2601753e+03],
       [-8.4559551e+03, -2.0567831e+02,  8.9729010e+02, ...,
        -1.9727728e+03,  5.9253208e+03,  5.6632790e+01],
       [-1.3929858e+02,  6.2342556e+01,  2.1850412e+01, ...,
        -1.2601753e+03,  5.6632790e+01,  4.5298849e+02]], dtype=float32)

In [44]:
#print(np.allclose(NTK_layerwise.cpu().detach().numpy(),NTK_easy))
print(np.allclose(NTK_layerwise.detach().numpy(),autograd_NTK,rtol=1e-1))
print(np.allclose(NTK_easy,autograd_NTK))

True


NameError: name 'NTK_easy' is not defined

# Compare Layerwise Components

In [49]:
print(layer_components_w1 @ layer_components_w1.T)
print(layer_components_w2 @ layer_components_w2.T)
print(layer_components_w3 @ layer_components_w3.T)
print(layer_components_w4 @ layer_components_w4.T)
print(layer_components_w5 @ layer_components_w5.T)
print(layer_components_w6 @ layer_components_w6.T)
print(layer_components_w7 @ layer_components_w7.T)
print(' ')
print(layer_components_b1 @ layer_components_b1.T)
print(layer_components_b2 @ layer_components_b2.T)
print(layer_components_b3 @ layer_components_b3.T)
print(layer_components_b4 @ layer_components_b4.T)
print(layer_components_b5 @ layer_components_b5.T)
print(layer_components_b6 @ layer_components_b6.T)
print(layer_components_b7 @ layer_components_b7.T)

NTK_components_autograd = [
    layer_components_w7 @ layer_components_w7.T,
    layer_components_b7 @ layer_components_b7.T,
    layer_components_w6 @ layer_components_w6.T,
    layer_components_b6 @ layer_components_b6.T,
    layer_components_w5 @ layer_components_w5.T,
    layer_components_b5 @ layer_components_b5.T,
    layer_components_w4 @ layer_components_w4.T,
    layer_components_b4 @ layer_components_b4.T,
    layer_components_w3 @ layer_components_w3.T,
    layer_components_b3 @ layer_components_b3.T,
    layer_components_w2 @ layer_components_w2.T,
    layer_components_b2 @ layer_components_b2.T,
    layer_components_w1 @ layer_components_w1.T,
    layer_components_b1 @ layer_components_b1.T,
]

[[ 8.8464969e+04  2.6263931e+03 -4.8651577e+03 ...  3.5916440e+03
  -8.4994512e+03  1.2480615e+02]
 [ 2.6263931e+03  6.2154340e+02  1.0863655e+02 ... -7.7489374e+02
  -9.8777687e+01  2.6040871e+01]
 [-4.8651577e+03  1.0863655e+02  1.3723626e+04 ... -1.9260972e+03
   9.8088300e+02 -1.0505713e+02]
 ...
 [ 3.5916440e+03 -7.7489374e+02 -1.9260972e+03 ...  2.2143219e+04
  -1.8547600e+03 -8.4317175e+02]
 [-8.4994512e+03 -9.8777687e+01  9.8088300e+02 ... -1.8547600e+03
   3.2994490e+03  1.0212484e+02]
 [ 1.2480615e+02  2.6040871e+01 -1.0505713e+02 ... -8.4317175e+02
   1.0212484e+02  2.4311635e+02]]
[[ 2.47159922e+04  1.38156631e+02 -6.25420593e+02 ... -1.28638721e+03
  -7.23600220e+02  1.76240921e+01]
 [ 1.38156631e+02  3.16876068e+02 -1.24793434e+02 ... -6.39673706e+02
   2.79912357e+01 -1.93850374e+00]
 [-6.25420593e+02 -1.24793434e+02  4.95354688e+03 ...  7.54058350e+02
  -2.47659531e+02  1.74559002e+01]
 ...
 [-1.28638721e+03 -6.39673706e+02  7.54058350e+02 ...  1.74292520e+04
  -1.32594

In [57]:
l = 6

print(np.allclose(NTK_components_autograd[l],components[l],1e-1))

print(NTK_components_autograd[l])
print(components[l])

True
[[ 2.99031885e+03  7.70762730e+00  6.38075409e+01 ... -2.13095490e+02
  -4.86535416e+01 -3.28437662e+00]
 [ 7.70762730e+00  5.56665001e+01 -1.86814251e+01 ... -1.24881077e+01
  -4.94960880e+00 -1.36203027e+00]
 [ 6.38075409e+01 -1.86814251e+01  5.12237427e+02 ... -3.37170601e+01
  -1.31761055e+01  6.47433758e-01]
 ...
 [-2.13095490e+02 -1.24881077e+01 -3.37170601e+01 ...  1.02425720e+03
   2.18924260e+00 -1.98265576e+00]
 [-4.86535416e+01 -4.94960880e+00 -1.31761055e+01 ...  2.18924260e+00
   1.75513916e+02 -1.12184620e+00]
 [-3.28437662e+00 -1.36203027e+00  6.47433758e-01 ... -1.98265576e+00
  -1.12184620e+00  7.63436508e+00]]
tensor([[ 2.9903e+03,  7.7076e+00,  6.3808e+01,  ..., -2.1310e+02,
         -4.8654e+01, -3.2844e+00],
        [ 7.7076e+00,  5.5667e+01, -1.8681e+01,  ..., -1.2488e+01,
         -4.9496e+00, -1.3620e+00],
        [ 6.3808e+01, -1.8681e+01,  5.1224e+02,  ..., -3.3717e+01,
         -1.3176e+01,  6.4744e-01],
        ...,
        [-2.1310e+02, -1.2488e+01, -3